In [161]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
import math




In [14]:
data_path = "./fds-link-prediction-madhura/archive"

In [15]:
df_attributes = pd.read_csv(f"{data_path}/attributes.csv")
df_predictions = pd.read_csv(f"{data_path}/attributes.csv")
df_solutionInput = pd.read_csv(f"{data_path}/solutionInput.csv")

In [16]:
#change data structure so it can be use in the model
solution_edges = df_solutionInput[['int1', 'int2']].astype(str).values.tolist()

In [17]:
# Load the graphs
G = nx.read_edgelist(f"{data_path}/edges_train.edgelist", delimiter=',', nodetype=str)

In [155]:
def getFeature(G, i, j):
    features = []
    
    # Preferential attachment
    pa = len(list(G.neighbors(i))) * len(list(G.neighbors(j)))
    features.append(pa)
    
    # Common neighbors
    common_neighbors = len(list(nx.common_neighbors(G, i, j)))
    features.append(common_neighbors)

    # Proportion of common neighbours
    proportion_common_neighbours = len(list(nx.common_neighbors(G, i, j))) / (len(list(G.neighbors(i))) + len(list(G.neighbors(j))))
    features.append(proportion_common_neighbours)

    # Jaccard similarity
    jaccard = list(nx.jaccard_coefficient(G, [(i, j)]))[0][2]
    features.append(jaccard)

    # Adamic-Adar index
    adamic_ajar = list(nx.adamic_adar_index(G, [(i, j)]))[0][2]
    features.append(adamic_ajar)

    # Resource allocation index 
    ra_index = list(nx.resource_allocation_index(G, [(i, j)]))[0][2]
    features.append(ra_index)

    # Resouece allocation index Soundarajan-Hopcroft
    rash_index = list(nx.ra_index_soundarajan_hopcroft(G, [(i, j)], 'community'))[0][2]
    features.append(rash_index)
    
    return np.array(features)

In [162]:
def getFeature(G, i, j):
    features = []
    
    # Store neighbor sets for reuse
    neighbors_i = set(G.neighbors(i))
    neighbors_j = set(G.neighbors(j))
    len_neighbors_i = len(neighbors_i)
    len_neighbors_j = len(neighbors_j)
    
    # Existing features
    # Preferential attachment
    pa = len_neighbors_i * len_neighbors_j
    features.append(pa)
    
    # Common neighbors
    common_neighbors_set = neighbors_i.intersection(neighbors_j)
    common_neighbors = len(common_neighbors_set)
    features.append(common_neighbors)

    # Proportion of common neighbours
    try:
        proportion_common_neighbours = common_neighbors / (len_neighbors_i + len_neighbors_j - common_neighbors)
    except ZeroDivisionError:
        proportion_common_neighbours = 0
    features.append(proportion_common_neighbours)

    # Jaccard similarity
    jaccard = list(nx.jaccard_coefficient(G, [(i, j)]))[0][2]
    features.append(jaccard)

    # Adamic-Adar index
    adamic_adar = list(nx.adamic_adar_index(G, [(i, j)]))[0][2]
    features.append(adamic_adar)

    # Resource allocation index 
    ra_index = list(nx.resource_allocation_index(G, [(i, j)]))[0][2]
    features.append(ra_index)

    # New features
    
    # 1. Shortest path length (if not connected)
    # try:
    #     shortest_path = nx.shortest_path_length(G, i, j)
    # except nx.NetworkXNoPath:
    #     shortest_path = -1  # or some other default value
    # features.append(shortest_path)
    
    # 2. Local clustering coefficients
    clustering_i = nx.clustering(G, i)
    clustering_j = nx.clustering(G, j)
    features.append(clustering_i)
    features.append(clustering_j)
    
    # 3. Degree centrality difference
    degree_centrality_diff = abs(nx.degree_centrality(G)[i] - nx.degree_centrality(G)[j])
    features.append(degree_centrality_diff)
    
    # 4. Average neighbor degree
    avg_neighbor_degree_i = sum(len(list(G.neighbors(n))) for n in neighbors_i) / len_neighbors_i if len_neighbors_i > 0 else 0
    avg_neighbor_degree_j = sum(len(list(G.neighbors(n))) for n in neighbors_j) / len_neighbors_j if len_neighbors_j > 0 else 0
    features.append(avg_neighbor_degree_i)
    features.append(avg_neighbor_degree_j)
    
    # 5. Common neighbor two-hop
    two_hop_neighbors_i = set().union(*[set(G.neighbors(n)) for n in neighbors_i]) if len_neighbors_i > 0 else set()
    two_hop_neighbors_j = set().union(*[set(G.neighbors(n)) for n in neighbors_j]) if len_neighbors_j > 0 else set()
    common_two_hop = len(two_hop_neighbors_i.intersection(two_hop_neighbors_j))
    features.append(common_two_hop)
    
    # 6. Cosine similarity of neighbor sets
    if len_neighbors_i > 0 and len_neighbors_j > 0:
        cosine_sim = common_neighbors / (math.sqrt(len_neighbors_i) * math.sqrt(len_neighbors_j))
    else:
        cosine_sim = 0
    features.append(cosine_sim)
    
    # 7. Hub and authority scores
    hub_auth = nx.hits(G)[0]
    hub_score_diff = abs(hub_auth[i] - hub_auth[j])
    features.append(hub_score_diff)
    
    return np.array(features)

In [163]:
def extract_edge_features(graph, edges):
    features = []
    for (i, j) in edges:
        edge_features = getFeature(graph, i, j)
        features.append(edge_features)
    return np.array(features)

In [164]:
# Prepare features for the edges in the data set
edges = list(G.edges())
X = extract_edge_features(G, edges)
y = [1] * len(edges)

In [165]:
X

array([[3.16800000e+03, 1.60000000e+01, 1.63265306e-01, ...,
        2.38000000e+02, 2.84267622e-01, 1.96612198e-03],
       [9.60000000e+02, 3.00000000e+00, 4.61538462e-02, ...,
        1.58000000e+02, 9.68245837e-02, 3.12608744e-03],
       [3.40800000e+03, 1.50000000e+01, 1.44230769e-01, ...,
        2.23000000e+02, 2.56945766e-01, 2.10118749e-03],
       ...,
       [3.50000000e+01, 2.00000000e+00, 2.00000000e-01, ...,
        3.60000000e+01, 3.38061702e-01, 5.79988225e-05],
       [2.10000000e+01, 1.00000000e+00, 1.11111111e-01, ...,
        1.90000000e+01, 2.18217890e-01, 1.04728736e-04],
       [5.00000000e+01, 0.00000000e+00, 0.00000000e+00, ...,
        1.10000000e+01, 0.00000000e+00, 1.00715401e-04]])

In [166]:
# Generate negative samples for the data set
def generate_negative_samples(graph, num_samples):
    nodes = list(graph.nodes())
    negative_samples = []
    while len(negative_samples) < num_samples:
        u, v = np.random.choice(nodes, size=2, replace=False)
        if not graph.has_edge(u, v):
            negative_samples.append((u, v))
    return negative_samples

num_negative_samples = int(len(edges)*1.00)  # % of negative samples
negative_edges = generate_negative_samples(G, num_negative_samples)
X_negative = extract_edge_features(G, negative_edges)
y_negative = [0] * len(negative_edges)

# Combine positive and negative samples for data set
X = np.vstack((X, X_negative))
y = np.concatenate((y, y_negative))

In [167]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [168]:
X_train

array([[1.62000000e+02, 0.00000000e+00, 0.00000000e+00, ...,
        7.50000000e+01, 0.00000000e+00, 4.57052642e-04],
       [5.00000000e+01, 0.00000000e+00, 0.00000000e+00, ...,
        2.60000000e+01, 0.00000000e+00, 1.15746189e-04],
       [2.40000000e+01, 0.00000000e+00, 0.00000000e+00, ...,
        1.40000000e+01, 0.00000000e+00, 1.63108613e-04],
       ...,
       [7.00000000e+01, 0.00000000e+00, 0.00000000e+00, ...,
        2.50000000e+01, 0.00000000e+00, 5.90660100e-04],
       [4.80000000e+01, 1.00000000e+00, 7.69230769e-02, ...,
        1.60000000e+01, 1.44337567e-01, 3.91658766e-06],
       [8.50000000e+01, 1.00000000e+00, 4.76190476e-02, ...,
        2.20000000e+01, 1.08465229e-01, 8.27593762e-04]])

In [216]:
clf = LogisticRegression(random_state=0, penalty='l2', C=1.5, solver='lbfgs', max_iter=2000)
clf.fit(X_train, y_train)

scores = cross_val_score(clf, X_test, y_test, cv=5)

print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))


0.88 accuracy with a standard deviation of 0.02


In [217]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest model
rf_clf = RandomForestClassifier(random_state=0, n_estimators=100, max_depth=10)
rf_clf.fit(X_train, y_train)

# Cross-validation
rf_scores = cross_val_score(rf_clf, X_test, y_test, cv=5)
print("Random Forest: %0.2f accuracy with a standard deviation of %0.2f" % (rf_scores.mean(), rf_scores.std()))

Random Forest: 0.90 accuracy with a standard deviation of 0.02


In [214]:
from sklearn.neighbors import KNeighborsClassifier

# KNN model
knn_clf = KNeighborsClassifier(n_neighbors= 20)
knn_clf.fit(X_train, y_train)

# Cross-validation
knn_scores = cross_val_score(knn_clf, X_test, y_test, cv=5)
print("KNN: %0.2f accuracy with a standard deviation of %0.2f" % (knn_scores.mean(), knn_scores.std()))

KNN: 0.89 accuracy with a standard deviation of 0.02


In [221]:
from sklearn.ensemble import GradientBoostingClassifier

# Gradient Boosting model
gb_clf = GradientBoostingClassifier(random_state=0, n_estimators=100, learning_rate=0.1, max_depth=3)
gb_clf.fit(X_train, y_train)

# Cross-validation
gb_scores = cross_val_score(gb_clf, X_test, y_test, cv=5)
print("Gradient Boosting: %0.2f accuracy with a standard deviation of %0.2f" % (gb_scores.mean(), gb_scores.std()))

Gradient Boosting: 0.90 accuracy with a standard deviation of 0.02


In [219]:

solution_edges = df_solutionInput[['int1', 'int2']].astype(str).values.tolist()
X_solution = extract_edge_features(G, solution_edges)

predictions = rf_clf.predict(X_solution)

df_solutionInput['prediction'] = predictions

#df_solutionInput.to_csv('predicted_solution.csv', index=False)

df_solutionInput.head()

num_positive_predictions = df_solutionInput['prediction'].sum()

print(f"Number of positive predictions: {num_positive_predictions}")


ValueError: X has 15 features, but RandomForestClassifier is expecting 14 features as input.

In [224]:
predictions = rf_clf.predict(X_solution)

df_solutionInput['prediction'] = predictions

#df_solutionInput.to_csv('predicted_solution.csv', index=False)

df_solutionInput.head()

num_positive_predictions = df_solutionInput['prediction'].sum()

print(f"Number of positive predictions: {num_positive_predictions}")

Number of positive predictions: 643


In [229]:
# Get predicted probabilities instead of class predictions
prediction_probabilities = knn_clf.predict_proba(X_solution)

# Get the probabilities for the positive class (class 1)
positive_class_probabilities = prediction_probabilities[:, 1]

# Find the threshold that gives exactly 50% positive predictions
# This is the median of the probabilities
threshold = np.median(positive_class_probabilities)

# Make predictions using this threshold
adjusted_predictions = (positive_class_probabilities >= threshold).astype(int)

# Update the predictions in your DataFrame
df_solutionInput['prediction'] = adjusted_predictions

In [230]:
df_solutionInput['prediction'].sum()

749

In [231]:
df_solutionInput[["prediction"]].to_csv("./attempt_8.csv")